# Predição de aprovação no ensino médio

### Features
* school : escola do estudante (binário: "GP" ou "MS")
* sex : sexo do estudante (binário: "F" - feminino ou "M" - masculino)
* age : idade do estudante (numérico: de 15 a 22)
* address : tipo de endereço do estudante (binário: "U" - urbano ou "R" - rural)
* famsize : tamanho da família (binário: "LE3" - até 3 pessoas ou "GT3" - mais de 3)
* Pstatus : situação de coabitação parental (binário: "T" - vivendo juntos ou "A" - separados)
* Medu : nível de escolaridade da mãe (numérico: 0 - nenhum, 1 - educação primária (até quarto ano), 2 - quinto a nono ano, 3 - ensino médio ou 4 - educação superior)
* Fedu : nível de escolaridade do pai (numérico: 0 - nenhum, 1 - educação primária (até quarto ano), 2 - quinto a nono ano, 3 - ensino médio ou 4 - educação superior)
* Mjob : profissão da mãe (nominal: "teacher" (professora), "health" (relacionado à saúde), "services" (ex. cargo administrativo ou policial), "at_home" (de casa) ou "other" (outros))
* Fjob : profissão do pai (nominal: "teacher" (professor), "health" (relacionado à saúde), "services" (ex. cargo administrativo ou policial), "at_home" (de casa) ou "other" (outros))
* reason : razão para escolher a escola (nominal: "home" (proximidade), "reputation" (reputação), "course" (preferência do curso) ou "other" (outro))
* guardian : responsável pelo estudante (nominal: "mother" (mãe), "father" (pai) ou "other" (outro))
* traveltime : tempo do trajeto casa-escola (numérico: 1 - <15 min., 2 - 15 a 30 min., 3 - 30 min. a 1 hora, ou 4 - >1 hora)
* studytime : tempo semanal de estudo (numérico: 1 - <2 horas, 2 - 2 a 5 horas, 3 - 5 a 10 horas, ou 4 - >10 horas)
* failures : número de reprovações passadas (numérico: n se 1<=n<3, senão 4)
* schoolsup : suporte educacional extra (binário: sim ou não)
* famsup : suporte educacional familiar (binário: sim ou não)
* paid : classes pagas extras dentre as matérias curriculares (Matemática ou Português) (binário: sim ou não)
* activities : atividades extra-curriculares (binário: sim ou não)
* nursery : frequentou educação infantil (binário: sim não)
* higher : pretende cursar educação superior (binário: sim ou não)
* internet : Acesso à internet em casa (binário: sim ou não)
* romantic : em um relacionamento afetivo (binário: sim ou não)
* famrel : qualidade das relações familiares (numérico: de 1 - muito ruim a 5 - excelente)
* freetime : tempo livre depois da escola (numérico: de 1 - muito baixo a 5 - muito alto)
* goout : frequência de saída com os amigos (numérico: de 1 - muito baixa a 5 - muito alta)
* Dalc : consumo de álcool durante dias úteis (numérico: de 1 - muito baixo a 5 - muito alto)
* Walc : consumo de álcool durante os fins de semana (numérico: de 1 - muito baixo a 5 - muito alto)
* health : estado atual de saúde (numérico: de 1 - muito ruim a 5 - muito bom)
* absences : número de faltas na escola (numérico: de 0 a 93)
* passed : se o estudante passou no exame final (binário: sim ou não)

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

### Carregando datase

In [2]:
dados_estudantes = pd.read_csv("student-data.csv")
print("Dados carregados com sucesso!")

Dados carregados com sucesso!


In [3]:
print(dados_estudantes)

    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
5       GP   M   16       U     LE3       T     4     3  services     other   
6       GP   M   16       U     LE3       T     2     2     other     other   
7       GP   F   17       U     GT3       A     4     4     other   teacher   
8       GP   M   15       U     LE3       A     3     2  services     other   
9       GP   M   15       U     GT3       T     3     4     other     other   
10      GP   F   15       U     GT3       T     4     4   teacher    health   
11      GP   F   15       U     GT3       T     2   

In [4]:
n_estudantes, n_features = dados_estudantes.shape
n_passed, _ = dados_estudantes[dados_estudantes['passed'] == 'yes'].shape
n_failed, _ = dados_estudantes[dados_estudantes['passed'] == 'no'].shape
indice_aprovacao = (n_passed / n_estudantes) * 100

print("Número de estudantes: {}".format(n_estudantes))
print("Número de features: {}".format(n_features))
print("Número de estudantes aprovados: {}".format(n_passed))
print("Número de estudantes reprovados: {}".format(n_failed))
print("Indice de aprovação: {:.2f}%".format(indice_aprovacao))

Número de estudantes: 395
Número de features: 31
Número de estudantes aprovados: 265
Número de estudantes reprovados: 130
Indice de aprovação: 67.09%


### Arrumando os dados
#### Separando atributos de variavéis alvo

In [5]:
feature_cols = list(dados_estudantes.columns[:-1])
target_cols = dados_estudantes.columns[-1] 

print("Colunas de atributos:\n{}".format(feature_cols))
print("\nColuna-alvo: {}".format(target_cols))

X_all = dados_estudantes[feature_cols]
y_all = dados_estudantes[target_cols]

Colunas de atributos:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Coluna-alvo: passed


#### Pré processamento de dados estudantis
 * Não numericos -> binários
 * Categorico    -> Variavéis Dummy

In [6]:
def preprocess_features(X):
    
    output = pd.DataFrame(index = X.index)
    for col, col_data in X.iteritems():
 
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
    
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print("{} features processadas".format(len(X_all.columns)))

48 features processadas


In [7]:
print(X_all)

     school_GP  school_MS  sex_F  sex_M  age  address_R  address_U  \
0            1          0      1      0   18          0          1   
1            1          0      1      0   17          0          1   
2            1          0      1      0   15          0          1   
3            1          0      1      0   15          0          1   
4            1          0      1      0   16          0          1   
5            1          0      0      1   16          0          1   
6            1          0      0      1   16          0          1   
7            1          0      1      0   17          0          1   
8            1          0      0      1   15          0          1   
9            1          0      0      1   15          0          1   
10           1          0      1      0   15          0          1   
11           1          0      1      0   15          0          1   
12           1          0      0      1   15          0          1   
13           1      

### Divisão de treinamento e de teste

In [8]:
X_all, y_all = shuffle(X_all, y_all, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, stratify=y_all, test_size=0.24, random_state=42)


print("O conjunto de treinamento tem {} amostras.".format(X_train.shape[0]))
print("O conjunto de teste tem {} amostras.".format(X_test.shape[0]))

print("\nVerificando balanceamento:")
print("Indice de aprovação no conjunto de treinamento: {:.2f}%".format(100 * (y_train == 'yes').mean()))
print("Indice de aprovação no conjunto de teste:       {:.2f}%".format(100 * (y_test == 'yes').mean()))

O conjunto de treinamento tem 300 amostras.
O conjunto de teste tem 95 amostras.

Verificando balanceamento:
Indice de aprovação no conjunto de treinamento: 67.00%
Indice de aprovação no conjunto de teste:       67.37%
